In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!pip install sentence-transformers

In [3]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.3 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login(token='<YOUR TOKEN HERE>')

In [ ]:
!pip install datasets

In [4]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer, util

In [5]:
wikipedia_text_corpus = load_dataset('rag-datasets/rag-mini-wikipedia', 'text-corpus')
wikipedia_question_answer = load_dataset('rag-datasets/rag-mini-wikipedia', 'question-answer')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/719 [00:00<?, ?B/s]

part.0.parquet:   0%|          | 0.00/797k [00:00<?, ?B/s]

Generating passages split:   0%|          | 0/3200 [00:00<?, ? examples/s]

part.0.parquet:   0%|          | 0.00/54.4k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/918 [00:00<?, ? examples/s]

In [8]:
wikipedia_text_corpus['passages'][0]

{'passage': 'Uruguay (official full name in  ; pron.  , Eastern Republic of  Uruguay) is a country located in the southeastern part of South America.  It is home to 3.3 million people, of which 1.7 million live in the capital Montevideo and its metropolitan area.',
 'id': 0}

In [11]:
wikipedia_question_answer['test']['question']

['Was Abraham Lincoln the sixteenth President of the United States?',
 'Did Lincoln sign the National Banking Act of 1863?',
 'Did his mother die of pneumonia?',
 "How many long was Lincoln's formal education?",
 'When did Lincoln begin his political career?',
 'What did The Legal Tender Act of 1862 establish?',
 'Who suggested Lincoln grow a beard?',
 'When did the Gettysburg address argue that America was born?',
 'Did Lincoln beat John C. Breckinridge in the 1860 election?',
 'Was Abraham Lincoln the first President of the United States?',
 'Did Lincoln start his political career in 1832?',
 'Did Lincoln ever represent Alton & Sangamon Railroad?',
 'Which county was Lincoln born in?',
 'When did Lincoln first serve as President?',
 'Who assassinated Lincoln?',
 'Did Lincoln win the election of 1860?',
 'Who was the general in charge at the Battle of Antietam?',
 'Why did Lincoln issue the Emancipation Proclamation?',
 'Do scholars rank lincoln among the top three presidents?',
 'Did

In [13]:
wikipedia_question_answer['test']['answer']

['yes',
 'yes',
 'no',
 '18 months',
 '1832',
 'the United States Note, the first paper currency in United States history',
 '11-year-old Grace Bedell',
 '1776',
 'yes',
 'No',
 'Yes',
 'Yes',
 'Hardin County',
 'March 4, 1861',
 'John Wilkes Booth',
 'Yes',
 'General McClellan',
 'To free slaves ',
 'Yes',
 'Yes',
 'Yes',
 'seven',
 '1846',
 '23 years',
 'he defended William "Duff" Armstrong',
 'Yes',
 'Yes',
 'Amedeo Avogadro',
 'Amedeo Avogadro',
 '1821',
 '1820',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'Avogadro had been recalled to Turin university',
 'Anders Celsius',
 'The Celsius crater on the Moon',
 '1733',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Anders Celsius',
 'named after him',
 'Yes',
 'Yes it had',
 'Yes',
 'Yes he did',
 'He was professor of astronomy at Uppsala University',
 'The scale was reversed',
 'Yes',
 'No',
 'Yes',
 'the head, the thorax, and the abdomen',
 'mimicry',
 'Colorado potato beetle',
 'their hardened, often darkened head, 

In [14]:
wikipedia_question_answer['test']['question'][0]

'Was Abraham Lincoln the sixteenth President of the United States?'

In [15]:
wikipedia_question_answer['test']['answer'][0]

'yes'

In [16]:
documents = wikipedia_text_corpus['passages']['passage']

In [17]:
documents

['Uruguay (official full name in  ; pron.  , Eastern Republic of  Uruguay) is a country located in the southeastern part of South America.  It is home to 3.3 million people, of which 1.7 million live in the capital Montevideo and its metropolitan area.',
 'It is bordered by Brazil to the north, by Argentina across the bank of both the Uruguay River to the west and the estuary of RÃ\xado de la Plata to the southwest, and the South Atlantic Ocean to the southeast. It is the second smallest independent country in South America, larger only than Suriname and the French overseas department of French Guiana.',
 'Montevideo was founded by the Spanish in the early 18th century as a military stronghold. Uruguay won its independence in 1828 following a three-way struggle between Spain, Argentina and Brazil. It is a constitutional democracy, where the president fulfills the roles of both head of state and head of government',
 'The economy is largely based in agriculture (making up 10% of the GDP

In [18]:
embeddings_model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
document_embeddings = embeddings_model.encode(documents, batch_size=64, show_progress_bar=True)

Batches:   0%|          | 0/50 [00:00<?, ?it/s]

In [24]:
document_embeddings[0]

array([ 6.98546087e-03, -6.14981316e-02, -6.68370277e-02, -8.28599744e-03,
        4.05003503e-02, -2.58229468e-02,  8.00510496e-02,  7.76856095e-02,
       -2.07005627e-03,  1.06492899e-01,  8.77064764e-02, -2.93755829e-02,
        3.29684503e-02, -2.88177617e-02,  2.90424842e-02, -2.61885822e-02,
       -2.37387605e-04, -5.43346815e-02,  5.99242896e-02, -5.19942082e-02,
        3.61571684e-02,  2.56331638e-03,  6.04414940e-02,  5.50898835e-02,
       -2.60236058e-02,  6.07446805e-02,  6.49650255e-03,  1.71787776e-02,
       -3.08122635e-02,  8.71879682e-02, -1.09186629e-02, -4.66562398e-02,
        2.76523717e-02,  5.97021580e-02, -1.20803285e-02,  1.87990777e-02,
       -2.58597564e-02, -5.76118566e-02,  5.18750697e-02, -2.53145974e-02,
       -1.55376084e-02,  7.17405602e-02,  1.35017629e-03, -3.15205716e-02,
        1.36287594e-02, -6.25519753e-02,  1.12760223e-01,  4.19790559e-02,
        4.09692638e-02,  3.12647270e-03,  7.86306486e-02, -1.26050431e-02,
       -3.44477147e-02, -

In [20]:
question = wikipedia_question_answer['test']['question'][0]
answer = wikipedia_question_answer['test']['answer'][0]

In [21]:
question

'Was Abraham Lincoln the sixteenth President of the United States?'

In [22]:
answer

'yes'

In [25]:
question_embedding = embeddings_model.encode(question, batch_size=64, show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
context = util.semantic_search(question_embedding, document_embeddings, top_k=3)

In [27]:
context

[[{'corpus_id': 288, 'score': 0.7095187902450562},
  {'corpus_id': 278, 'score': 0.5840358734130859},
  {'corpus_id': 697, 'score': 0.5568780303001404}]]

In [28]:
documents[288]

'Young Abraham Lincoln'

In [29]:
documents[278]

'Abraham Lincoln (February 12, 1809 â\x80\x93 April 15, 1865) was the sixteenth President of the United States, serving from March 4, 1861 until his assassination. As an outspoken opponent of the expansion of slavery in the United States, "[I]n his short autobiography written for the 1860 presidential campaign, Lincoln would describe his protest in the Illinois legislature as one that \'briefly defined his position on the slavery question, and so far as it goes, it was then the same that it is now." This was in reference to the anti-expansion sentiments he had then expressed. Doris Kearns Goodwin, Team of Rivals: The Political Genius of Abraham Lincoln (2005) p. 91.  Holzer pg. 232.  Writing of the Cooper Union  speech, Holzer notes, "Cooper Union proved a unique confluence of political culture, rhetorical opportunity, technological innovation, and human genius, and it brought Abraham Lincoln to the center stage of American politics at precisely the right time and place, and with preci

In [30]:
documents[697]

'Sixteen months before his death, his son, John Quincy Adams, became the sixth President of the United States (1825 1829), the only son of a former President to hold the office until George W. Bush in 2001.'

In [31]:
context_doc_ids = [c['corpus_id'] for c in context[0]]

In [32]:
context_doc_ids

[288, 278, 697]

In [33]:
context_docs = [documents[i] for i in context_doc_ids]

In [34]:
context_docs

['Young Abraham Lincoln',
 'Abraham Lincoln (February 12, 1809 â\x80\x93 April 15, 1865) was the sixteenth President of the United States, serving from March 4, 1861 until his assassination. As an outspoken opponent of the expansion of slavery in the United States, "[I]n his short autobiography written for the 1860 presidential campaign, Lincoln would describe his protest in the Illinois legislature as one that \'briefly defined his position on the slavery question, and so far as it goes, it was then the same that it is now." This was in reference to the anti-expansion sentiments he had then expressed. Doris Kearns Goodwin, Team of Rivals: The Political Genius of Abraham Lincoln (2005) p. 91.  Holzer pg. 232.  Writing of the Cooper Union  speech, Holzer notes, "Cooper Union proved a unique confluence of political culture, rhetorical opportunity, technological innovation, and human genius, and it brought Abraham Lincoln to the center stage of American politics at precisely the right tim

In [35]:
prompt = f'Answer the following question using the provided context. Provide the answer ' \
         f'using this template: <ANSWER>your answer here</ANSWER>. \n\n' \
         f'Context:\n' \
         f'Document 1: {context_docs[0]}\n' \
         f'Document 2: {context_docs[1]}\n' \
         f'Document 3: {context_docs[2]}\n\n' \
         f'Question: {question}'

In [36]:
print(prompt)

Answer the following question using the provided context. Provide the answer using this template: <ANSWER>your answer here</ANSWER>. 

Context:
Document 1: Young Abraham Lincoln
Document 2: Abraham Lincoln (February 12, 1809 â April 15, 1865) was the sixteenth President of the United States, serving from March 4, 1861 until his assassination. As an outspoken opponent of the expansion of slavery in the United States, "[I]n his short autobiography written for the 1860 presidential campaign, Lincoln would describe his protest in the Illinois legislature as one that 'briefly defined his position on the slavery question, and so far as it goes, it was then the same that it is now." This was in reference to the anti-expansion sentiments he had then expressed. Doris Kearns Goodwin, Team of Rivals: The Political Genius of Abraham Lincoln (2005) p. 91.  Holzer pg. 232.  Writing of the Cooper Union  speech, Holzer notes, "Cooper Union proved a unique confluence of political culture, rhetorical 

In [37]:
model_name = 'llama-2-7b-chat-hf'
tokenizer_name = 'llama-2-7b-chat-hf'
padding_size = 1024

In [38]:
tokenizer = AutoTokenizer.from_pretrained(f'meta-llama/{model_name}',
                                          use_auth_token=True,
                                          model_max_length=padding_size)
tokenizer.pad_token = tokenizer.bos_token
tokenizer.padding_side = 'left'

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:809: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [39]:
inputs = tokenizer(prompt, truncation=True, padding=True, return_tensors='pt')

In [40]:
input_ids = inputs['input_ids']
attention_masks = inputs['attention_mask']

In [41]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16,
                                         bnb_4bit_quant_type='nf4')

In [42]:
model = AutoModelForCausalLM.from_pretrained(f'meta-llama/{model_name}',
                                             device_map='cuda:0',
                                             quantization_config=quantization_config)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [43]:
output_ids = model.generate(input_ids=input_ids,
                            attention_mask=attention_masks)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [45]:
generated_answer = tokenizer.decode(output_ids[0],
                                    skip_special_tokens=True,
                                    clean_up_tokenization_spaces=True)

In [46]:
print(generated_answer)

Answer the following question using the provided context. Provide the answer using this template: <ANSWER>your answer here</ANSWER>. 

Context:
Document 1: Young Abraham Lincoln
Document 2: Abraham Lincoln (February 12, 1809 â April 15, 1865) was the sixteenth President of the United States, serving from March 4, 1861 until his assassination. As an outspoken opponent of the expansion of slavery in the United States, "[I]n his short autobiography written for the 1860 presidential campaign, Lincoln would describe his protest in the Illinois legislature as one that 'briefly defined his position on the slavery question, and so far as it goes, it was then the same that it is now." This was in reference to the anti-expansion sentiments he had then expressed. Doris Kearns Goodwin, Team of Rivals: The Political Genius of Abraham Lincoln (2005) p. 91.  Holzer pg. 232.  Writing of the Cooper Union  speech, Holzer notes, "Cooper Union proved a unique confluence of political culture, rhetorical 

In [47]:
answer

'yes'